# Loading Snowflake from Azure Blob with the Python SDK

This example loads a snowflake table from Azure blob storage with the Python SDK. We abstract some of the workings like handling query files with a simple wrapper class.

### Create snowflake wrapper class

In [2]:
#%%writefile ./snowq.py
import snowflake.connector
import pandas as pd

class snowQ(object):
    def __init__(self, user: str, password: str, account: str):
        self.user = user
        self.account = account
        self.ctx = snowflake.connector.connect(user=user,
                                               password=password,
                                               account=account)
        cs = self.ctx.cursor()
        try:
            cs.execute("SELECT current_version()")
            one_row = cs.fetchone()
            self.version = one_row[0]
        finally:
            cs.close()
    
    def __del__(self):
        self.ctx.close()
    
    # Execute Query
    def sql(self, query: str, timeout=10):
        cs = self.ctx.cursor()
        try:
            cs.execute(query, timeout=timeout)
        except Exception as e:
            if e.errno == 604:
                print("Query Timeout, rolling back")
                cs.execute("rollback")
            else:
                raise e
        return cs
    
    # Execute Query providing a SQL file that holds the Query
    def sqlf(self, filename: str, timeout=10):
        cs = self.ctx.cursor()

        if len(filename) > 0:
            with open(filename, 'r') as file:
                query = file.read().replace('\n', '')
                file.close()
        return self.sql(query, timeout)

Overwriting ./snowq.py


### Load Snowflake wrapper class

In [3]:
import snowflake.connector
#from snowq import snowQ    # Import the wrapper

keyvault = ws.get_default_keyvault()
snow=snowQ(user='XXXXX', password='XXXXX', account='XXXXX.west-europe.azure')

In [4]:
snow.version

'4.11.2'

### Create a file with the desired query

In [1]:
%%writefile ./load_azure_customers.sql
copy into "SNOWPIPE_DEMO"."PUBLIC"."CUSTOMERS"
  from 'azure://dataeu.blob.core.windows.net/snowflake/sales/customers'
  credentials=(azure_sas_token='?XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
  file_format = (type = csv field_delimiter = ',' skip_header = 1, FIELD_OPTIONALLY_ENCLOSED_BY='"')

Overwriting ./load_azure_customers.sql


In [6]:
c = snow.sql('TRUNCATE TABLE "SNOWPIPE_DEMO"."PUBLIC"."CUSTOMERS"')
c.fetchone()[0]

'Statement executed successfully.'

In [7]:
# Truncate the table and show results
snow.sql('TRUNCATE TABLE "SNOWPIPE_DEMO"."PUBLIC"."CUSTOMERS"').fetchone()[0]

'Statement executed successfully.'

In [8]:
# Load files from using the query in the file
snow.sqlf('load_azure_customers.sql').fetchone()[0]

'azure://dataeu.blob.core.windows.net/snowflake/sales/customers'

In [9]:
snow.sql('select * from "SNOWPIPE_DEMO"."PUBLIC"."CUSTOMERS" ORDER BY CUSTOMERID;').fetch_pandas_all()

,CUSTOMERID,CUSTOMERNAME
0,1,Tailspin Toys (Head Office)
1,2,"Tailspin Toys (Sylvanite, MT)"
2,3,"Tailspin Toys (Peeples Valley, AZ)"
3,4,"Tailspin Toys (Medicine Lodge, KS)"
4,5,"Tailspin Toys (Gasport, NY)"
...,...,...
658,1057,Ganesh Majumdar
659,1058,Jaroslav Fisar
660,1059,Jibek Juniskyzy
661,1060,Anand Mudaliyar
